In [31]:
#basic
import numpy as np, pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from tqdm import tqdm_notebook
from sklearn.decomposition import PCA
#Pretreatment
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.feature_selection import VarianceThreshold
#model
import lightgbm as lgb
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn import svm, neighbors, linear_model, neural_network
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.svm import NuSVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB
from sklearn.ensemble import RandomForestClassifier
import warnings
warnings.filterwarnings('ignore')

In [25]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [26]:
features = [c for c in train.columns if c not in ['id', 'target']]
train['target']=1
test['target']=0
data = pd.concat([train,test], axis=0)

In [27]:
lgb_params = {
    'metric': 'auc',
    'verbose': 1,
    'learning_rate': 0.05,
    'num_leaves': 31,
    "seed": 42
}

In [32]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state= 42)
oof = np.zeros(len(data))
val_aucs = []
for fold, (trn_idx, val_idx) in enumerate(skf.split(data, data['target'])):
    X_train, y_train = data.iloc[trn_idx][features], data.iloc[trn_idx]['target']
    X_valid, y_valid = data.iloc[val_idx][features], data.iloc[val_idx]['target']
    trn_data = lgb.Dataset(X_train, label=y_train)
    val_data = lgb.Dataset(X_valid, label=y_valid)
    evals_result = {}
    lgb_clf = lgb.train(lgb_params,
                        trn_data,
                        valid_sets=[val_data],
                        early_stopping_rounds=100,
                        verbose_eval=50,
                        evals_result=evals_result)

    p_valid = lgb_clf.predict(X_valid[features], num_iteration=lgb_clf.best_iteration)

    oof[val_idx] = p_valid
    val_score = roc_auc_score(y_valid, p_valid)
    val_aucs.append(val_score)

Training until validation scores don't improve for 100 rounds.
[50]	valid_0's auc: 0.499601
[100]	valid_0's auc: 0.498945
Did not meet early stopping. Best iteration is:
[3]	valid_0's auc: 0.50079
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's auc: 0.495285
[100]	valid_0's auc: 0.497333
Did not meet early stopping. Best iteration is:
[4]	valid_0's auc: 0.50247
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's auc: 0.500114
[100]	valid_0's auc: 0.499871
Did not meet early stopping. Best iteration is:
[26]	valid_0's auc: 0.503064
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's auc: 0.497789
[100]	valid_0's auc: 0.497222
Did not meet early stopping. Best iteration is:
[28]	valid_0's auc: 0.500756
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's auc: 0.495796
[100]	valid_0's auc: 0.494027
Did not meet early stopping. Best iteration is:
[1]	valid_0's auc: 0.499819


In [39]:
print("평균 ACU :",np.round(np.mean(val_aucs),3))
print("전체 ACU :",np.round( roc_auc_score(data['target'], oof),3))

평균 ACU : 0.501
전체 ACU : 0.501


# 결론
- 훈련데이터와 테스트데이터는 같은 분포에서 나왔다